# Deep Hedging in Incomplete Markets — GBM + Heston

**MSc Thesis Experiment Runner**

Runs the full deep hedging pipeline under **two market models**:
- **GBM** (constant volatility, calibrated to S&P 500)
- **Heston** (stochastic volatility, calibrated to S&P 500 / CBOE VIX)

**Models:** FNN Cone (sigmoid allocation), GRU (direct positions), OLS Regression (direct positions)

**Features:** FNN uses base + signature features (level 3, feat_dim=12). GRU/Regression use base features only (feat_dim=3).

## First Run
1. **Runtime → Change runtime type → A100 GPU** (Pro+ recommended)
2. Run **Cell 1** (clone + install)
3. Run **Cell 2** (fresh start — clears Drive + local outputs)
4. Run **Cell 4** (sanity check — tests)
5. Run **Cell 5** (quick test) OR **Cells 6–11** (full run, one model per cell)

## Resume After Disconnect
1. **Runtime → Change runtime type → A100 GPU**
2. Run **Cell 1** (clone + install)
3. **Skip Cell 2** — run **Cell 3** instead (restores saved outputs from Drive)
4. Run **Cell 4** (sanity check)
5. Re-run the model cell that was interrupted — it starts that model fresh while keeping all completed models' results. Then continue with remaining cells.

## Full Run — Sequential Model Cells (Cells 6–11)
Each model runs as a **separate subprocess** so GPU/CPU memory is fully freed between them. After each cell, outputs are backed up to Google Drive. Completed models are automatically skipped on re-run.

| Cell | Market | Model |
|------|--------|-------|
| 6 | GBM | Regression |
| 7 | GBM | GRU |
| 8 | GBM | FNN |
| 9 | Heston | Regression |
| 10 | Heston | GRU |
| 11 | Heston | FNN |

The last cell for each market generates comparison plots using saved results from all prior model runs.

In [ ]:
# Cell 1: Clone repo and install dependencies
!git clone https://github.com/thabangTheActuaryCoder/deep-hedging-thesis.git
%cd deep-hedging-thesis
!pip install -q torch numpy matplotlib optuna sqlalchemy scipy iisignature

import torch
print(f'\nPython: {__import__("sys").version}')
print(f'PyTorch: {torch.__version__}')
print(f'GPU available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'Device: {torch.cuda.get_device_name(0)}')
    mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f'Memory: {mem:.1f} GB')

In [ ]:
# Cell 2: FRESH START ONLY — clears ALL previous outputs (Drive + local)
# ⚠️ Skip this cell when resuming after a disconnect! Use Cell 3 instead.
import shutil, os

from google.colab import drive
drive.mount('/content/drive')

DRIVE_BACKUP = '/content/drive/MyDrive/deep_hedging_outputs'
if os.path.exists(DRIVE_BACKUP):
    shutil.rmtree(DRIVE_BACKUP)
    print(f'Cleared previous outputs from Google Drive ({DRIVE_BACKUP})')
else:
    print(f'Google Drive clean — no previous outputs found.')

LOCAL_OUTPUTS = '/content/deep-hedging-thesis/outputs'
if os.path.exists(LOCAL_OUTPUTS):
    shutil.rmtree(LOCAL_OUTPUTS)
    print(f'Cleared local outputs ({LOCAL_OUTPUTS})')

print('Ready for a fresh experiment run.')

In [ ]:
# Cell 3: RESUME AFTER DISCONNECT — restores saved outputs from Google Drive
# Run this instead of Cell 2 when reconnecting after a runtime timeout/crash.
# Completed models will be skipped automatically; interrupted models retrain from scratch.
import shutil, os

from google.colab import drive
drive.mount('/content/drive')

DRIVE_BACKUP = '/content/drive/MyDrive/deep_hedging_outputs'
LOCAL_OUTPUTS = '/content/deep-hedging-thesis/outputs'

if os.path.exists(DRIVE_BACKUP):
    # Restore Drive backup to local outputs
    if os.path.exists(LOCAL_OUTPUTS):
        shutil.rmtree(LOCAL_OUTPUTS)
    shutil.copytree(DRIVE_BACKUP, LOCAL_OUTPUTS)

    # Count what's been completed
    for market in ['gbm', 'heston']:
        ckpt_dir = os.path.join(LOCAL_OUTPUTS, market, 'checkpoints')
        if os.path.exists(ckpt_dir):
            ckpts = [f for f in os.listdir(ckpt_dir) if f.endswith('.pt')]
            models_done = [f.replace('_seed0.pt', '') for f in ckpts]
            print(f'  {market.upper()}: restored {len(ckpts)} checkpoints — {models_done}')
        else:
            print(f'  {market.upper()}: no checkpoints found')

    print(f'\nRestored outputs from Google Drive. Re-run the interrupted model cell to continue.')
else:
    print('No previous outputs found on Google Drive. Run Cell 2 for a fresh start.')

In [ ]:
# Cell 4: Sanity check — all tests should pass
!python -m pytest tests/test_validation.py -v

In [ ]:
# Cell 5 (QUICK TEST): ~10 min on A100, verifies both GBM + Heston pipelines
!python run_experiment.py --quick --market_model both

In [ ]:
# Cell 6 (GBM — Regression): Closed-form OLS, ~2 min
!python run_experiment.py --models regression --market_model gbm \
    --paths 100000 --epochs 1000 --patience 15 --batch_size 2048 --seeds 0

# Backup to Google Drive (survives runtime disconnects)
!cp -r outputs/ /content/drive/MyDrive/deep_hedging_outputs/
print('Backed up to Google Drive.')

In [ ]:
# Cell 7 (GBM — GRU): Optuna HP search + seed robustness
!python run_experiment.py --models gru --market_model gbm \
    --paths 100000 --epochs 1000 --patience 15 --batch_size 2048 --n_trials 60 --seeds 0

# Backup to Google Drive (survives runtime disconnects)
!cp -r outputs/ /content/drive/MyDrive/deep_hedging_outputs/
print('Backed up to Google Drive.')

In [ ]:
# Cell 8 (GBM — FNN): Optuna HP search + seed robustness (signature features)
# Last GBM cell — generates comparison plots with all 3 models
!python run_experiment.py --models fnn --market_model gbm \
    --paths 100000 --epochs 1000 --patience 15 --batch_size 2048 --n_trials 60 --seeds 0

# Backup to Google Drive (survives runtime disconnects)
!cp -r outputs/ /content/drive/MyDrive/deep_hedging_outputs/
print('Backed up to Google Drive.')

In [ ]:
# Cell 9 (Heston — Regression): Closed-form OLS, ~2 min
!python run_experiment.py --models regression --market_model heston \
    --paths 100000 --epochs 1000 --patience 15 --batch_size 2048 --seeds 0

# Backup to Google Drive (survives runtime disconnects)
!cp -r outputs/ /content/drive/MyDrive/deep_hedging_outputs/
print('Backed up to Google Drive.')

In [ ]:
# Cell 10 (Heston — GRU): Optuna HP search + seed robustness
!python run_experiment.py --models gru --market_model heston \
    --paths 100000 --epochs 1000 --patience 15 --batch_size 2048 --n_trials 60 --seeds 0

# Backup to Google Drive (survives runtime disconnects)
!cp -r outputs/ /content/drive/MyDrive/deep_hedging_outputs/
print('Backed up to Google Drive.')

In [ ]:
# Cell 11 (Heston — FNN): Optuna HP search + seed robustness (signature features)
# Last Heston cell — generates comparison plots with all 3 models
!python run_experiment.py --models fnn --market_model heston \
    --paths 100000 --epochs 1000 --patience 15 --batch_size 2048 --n_trials 60 --seeds 0

# Backup to Google Drive (survives runtime disconnects)
!cp -r outputs/ /content/drive/MyDrive/deep_hedging_outputs/
print('Backed up to Google Drive.')

In [ ]:
# Cell 12: Preview validation + test plots (both markets)
from IPython.display import Image, display
import glob

for market in ['gbm', 'heston']:
    for split, folder in [('Validation', 'plots_val'), ('Test', 'plots_test')]:
        imgs = sorted(glob.glob(f'outputs/{market}/{folder}/*.png'))
        if imgs:
            print(f'\n=== {market.upper()} — {split} Plots ===')
            for img in imgs:
                print(f'\n--- {img} ---')
                display(Image(filename=img, width=700))
        else:
            print(f'\n{market.upper()} — No {split.lower()} plots found yet.')

In [ ]:
# Cell 13: 3D hedge surface plots
from IPython.display import Image, display
import glob

for market in ['gbm', 'heston']:
    print(f'\n=== {market.upper()} — 3D Hedge Surface Plots ===')
    imgs = sorted(glob.glob(f'outputs/{market}/plots_3d/*.png'))
    if imgs:
        for img in imgs:
            print(f'\n--- {img} ---')
            display(Image(filename=img, width=700))
    else:
        print(f'No PNG 3D plots found for {market}. Check outputs/{market}/plots_3d/ for HTML files.')

In [ ]:
# Cell 14: Show validation + test metrics summary (both markets)
import json, os

for market in ['gbm', 'heston']:
    path = f'outputs/{market}/metrics_summary.json'
    if not os.path.exists(path):
        print(f'{market.upper()}: No metrics found. Run the experiment first.')
        continue

    with open(path) as f:
        summary = json.load(f)

    best_val = summary.get('best_model_val', summary.get('best_model', '?'))
    best_test = summary.get('best_model_test', '?')

    print(f'\n{"="*60}')
    print(f'  {market.upper()} — Best model (val): {best_val}  |  Best model (test): {best_test}')
    print(f'{"="*60}')

    # Validation metrics
    agg = summary.get('aggregated_val_metrics', {})
    if agg:
        print(f'\n  Validation Set:')
        for model, metrics in agg.items():
            mae = metrics.get('MAE', {})
            mse = metrics.get('MSE', {})
            print(f'    {model:12s}  MAE = {mae.get("mean",0):.6f} +/- {mae.get("std",0):.6f}  '
                  f'MSE = {mse.get("mean",0):.6f} +/- {mse.get("std",0):.6f}')

    # Test metrics
    test_agg = summary.get('aggregated_test_metrics', {})
    if test_agg:
        print(f'\n  Test Set:')
        for model, metrics in test_agg.items():
            mae = metrics.get('MAE', {})
            mse = metrics.get('MSE', {})
            print(f'    {model:12s}  MAE = {mae.get("mean",0):.6f} +/- {mae.get("std",0):.6f}  '
                  f'MSE = {mse.get("mean",0):.6f} +/- {mse.get("std",0):.6f}')

    print(f'\n  Best configs:')
    for model, cfg in summary.get('best_configs', {}).items():
        print(f'    {model}: {cfg}')

In [ ]:
# Cell 15: Show CSV metrics tables (both markets, val + test)
import os

for market in ['gbm', 'heston']:
    for split, fname in [('Validation', 'val_metrics_summary.csv'),
                         ('Test', 'test_metrics_summary.csv')]:
        csv_path = f'outputs/{market}/{fname}'
        if os.path.exists(csv_path):
            print(f'\n=== {market.upper()} — {split} ===')
            with open(csv_path) as f:
                print(f.read())
        else:
            print(f'{market.upper()} — {split}: No CSV summary found.')

## Model Comparison: Histograms + Violin Plots (GBM vs Heston)

**7 comparison figures** using real experiment terminal errors:
- **9a**: Terminal error histogram overlay per market (3 models overlaid)
- **9b**: Violin + box plot of terminal errors per market
- **9c**: Cross-market violin — same model, GBM vs Heston side-by-side
- **9d**: Cross-market histogram overlay per model
- **9e**: Shortfall distribution violin with CVaR95 annotations
- **9f**: Grouped bar chart — MAE, Mean Shortfall, P(V_T >= H) across markets
- **9g**: Grand combined violin — all 6 model-market combinations

In [ ]:
# Cell 17: Load terminal errors from experiment outputs
import json, os, numpy as np
import matplotlib.pyplot as plt

def load_experiment_errors():
    """Load terminal errors from experiment outputs (GBM and Heston).
    Falls back to simulated data if experiment hasn't run yet.
    """
    cross_path = os.path.join(os.getcwd(), 'outputs', 'cross_market', 'cross_market_errors.json')

    if os.path.exists(cross_path):
        with open(cross_path) as f:
            raw = json.load(f)
        data = {}
        for mkt, models in raw.items():
            data[mkt] = {m: np.array(e) for m, e in models.items()}
        return data

    # Fallback: generate representative simulated data
    print('No experiment outputs found — using simulated representative data.')
    np.random.seed(2026)
    n = 2000
    data = {
        'gbm': {
            'FNN': np.random.normal(0.02, 0.08, n),
            'GRU': np.random.normal(0.01, 0.04, n),
            'Regression': np.random.normal(0.03, 0.10, n),
        },
        'heston': {
            'FNN': np.random.normal(0.01, 0.12, n),
            'GRU': np.random.normal(0.005, 0.06, n),
            'Regression': np.random.normal(0.02, 0.15, n),
        },
    }
    return data

errors = load_experiment_errors()
models = list(errors.get('gbm', {}).keys())
markets = list(errors.keys())
print(f'Markets: {markets}')
print(f'Models: {models}')
for mkt in markets:
    for m in models:
        e = errors[mkt][m]
        print(f'  {mkt}/{m}: n={len(e)}, mean={e.mean():.4f}, std={e.std():.4f}')

In [ ]:
# Cell 18: Generate all 7 comparison plots (histogram + violin)
def plot_all_comparisons(errors, save_dir='Figures'):
    """Generate all histogram + violin comparison plots."""
    os.makedirs(save_dir, exist_ok=True)
    models = list(errors.get('gbm', errors.get('heston', {})).keys())
    markets = list(errors.keys())

    c_model = {'FNN': '#5C6BC0', 'GRU': '#26A69A', 'Regression': '#FF8F00'}
    c_market = {'gbm': '#1565C0', 'heston': '#8E24AA'}

    # ── FIGURE 9a: Terminal Error Histogram Overlay (per market) ──
    fig, axes = plt.subplots(1, len(markets), figsize=(9*len(markets), 6), sharey=True)
    if len(markets) == 1: axes = [axes]
    for ax, mkt in zip(axes, markets):
        for m in models:
            e = errors[mkt][m]
            ax.hist(e, bins=60, density=True, alpha=0.4, color=c_model[m], label=m)
            ax.axvline(e.mean(), color=c_model[m], linestyle='--', linewidth=1.5, alpha=0.8)
        ax.axvline(0, color='black', linewidth=1.5, label='$V_T = \\tilde{H}$')
        ax.set_xlabel('Terminal Error $e_T = V_T - \\tilde{H}$', fontsize=11)
        ax.set_ylabel('Density', fontsize=11)
        ax.set_title(f'{mkt.upper()}: Terminal Error Distribution', fontsize=13, fontweight='bold')
        ax.legend(fontsize=9); ax.grid(True, alpha=0.3)
    fig.suptitle('(9a) Terminal Error Histogram: Model Comparison', fontsize=15, fontweight='bold', y=1.02)
    fig.tight_layout()
    fig.savefig(os.path.join(save_dir, 'comparison_histogram_by_model.png'), dpi=200, bbox_inches='tight')
    plt.show()

    # ── FIGURE 9b: Violin Plot — Terminal Errors by Model (per market) ──
    fig, axes = plt.subplots(1, len(markets), figsize=(8*len(markets), 6), sharey=True)
    if len(markets) == 1: axes = [axes]
    for ax, mkt in zip(axes, markets):
        data_list = [errors[mkt][m] for m in models]
        parts = ax.violinplot(data_list, positions=range(len(models)), showmeans=True,
                             showmedians=True, showextrema=False)
        for i, pc in enumerate(parts['bodies']):
            pc.set_facecolor(c_model[models[i]]); pc.set_alpha(0.6)
        parts['cmeans'].set_color('#C62828'); parts['cmedians'].set_color('#1B5E20')
        bp = ax.boxplot(data_list, positions=range(len(models)), widths=0.15,
                       patch_artist=True, showfliers=False, zorder=5)
        for i, patch in enumerate(bp['boxes']):
            patch.set_facecolor(c_model[models[i]]); patch.set_alpha(0.8)
        ax.axhline(0, color='black', linewidth=1, linestyle='--', alpha=0.5)
        ax.set_xticks(range(len(models))); ax.set_xticklabels(models, fontsize=11)
        ax.set_ylabel('Terminal Error $e_T$', fontsize=11)
        ax.set_title(f'{mkt.upper()}: Violin + Box Plot', fontsize=13, fontweight='bold')
        ax.grid(True, alpha=0.3, axis='y')
    fig.suptitle('(9b) Terminal Error Distribution: Violin Plots', fontsize=15, fontweight='bold', y=1.02)
    fig.tight_layout()
    fig.savefig(os.path.join(save_dir, 'comparison_violin_by_model.png'), dpi=200, bbox_inches='tight')
    plt.show()

    # ── FIGURE 9c: Cross-Market Violin — Same model, GBM vs Heston ──
    if len(markets) >= 2:
        fig, axes = plt.subplots(1, len(models), figsize=(7*len(models), 6), sharey=True)
        if len(models) == 1: axes = [axes]
        for ax, m in zip(axes, models):
            data_list = [errors[mkt][m] for mkt in markets]
            parts = ax.violinplot(data_list, positions=range(len(markets)), showmeans=True,
                                 showmedians=True, showextrema=False)
            for i, pc in enumerate(parts['bodies']):
                pc.set_facecolor(c_market[markets[i]]); pc.set_alpha(0.6)
            parts['cmeans'].set_color('#C62828'); parts['cmedians'].set_color('#1B5E20')
            bp = ax.boxplot(data_list, positions=range(len(markets)), widths=0.15,
                           patch_artist=True, showfliers=False, zorder=5)
            for i, patch in enumerate(bp['boxes']):
                patch.set_facecolor(c_market[markets[i]]); patch.set_alpha(0.8)
            ax.axhline(0, color='black', linewidth=1, linestyle='--', alpha=0.5)
            ax.set_xticks(range(len(markets)))
            ax.set_xticklabels([mk.upper() for mk in markets], fontsize=11)
            ax.set_ylabel('Terminal Error $e_T$', fontsize=11)
            ax.set_title(f'{m}: GBM vs Heston', fontsize=13, fontweight='bold')
            ax.grid(True, alpha=0.3, axis='y')
        fig.suptitle('(9c) Cross-Market Comparison: GBM vs Heston per Model',
                     fontsize=15, fontweight='bold', y=1.02)
        fig.tight_layout()
        fig.savefig(os.path.join(save_dir, 'comparison_violin_cross_market.png'), dpi=200, bbox_inches='tight')
        plt.show()

    # ── FIGURE 9d: Cross-Market Histogram Overlay (per model) ──
    if len(markets) >= 2:
        fig, axes = plt.subplots(1, len(models), figsize=(7*len(models), 5), sharey=True)
        if len(models) == 1: axes = [axes]
        for ax, m in zip(axes, models):
            for mkt in markets:
                e = errors[mkt][m]
                ax.hist(e, bins=50, density=True, alpha=0.45, color=c_market[mkt],
                       label=f'{mkt.upper()} (mean={e.mean():.3f})')
            ax.axvline(0, color='black', linewidth=1.5)
            ax.set_xlabel('Terminal Error $e_T$', fontsize=11)
            ax.set_ylabel('Density', fontsize=11)
            ax.set_title(f'{m}', fontsize=13, fontweight='bold')
            ax.legend(fontsize=9); ax.grid(True, alpha=0.3)
        fig.suptitle('(9d) Cross-Market Histogram: GBM vs Heston per Model',
                     fontsize=15, fontweight='bold', y=1.02)
        fig.tight_layout()
        fig.savefig(os.path.join(save_dir, 'comparison_histogram_cross_market.png'), dpi=200, bbox_inches='tight')
        plt.show()

    # ── FIGURE 9e: Shortfall Distribution Violin (per market) ──
    fig, axes = plt.subplots(1, len(markets), figsize=(8*len(markets), 6), sharey=True)
    if len(markets) == 1: axes = [axes]
    for ax, mkt in zip(axes, markets):
        shortfalls = [np.maximum(-errors[mkt][m], 0) for m in models]
        parts = ax.violinplot(shortfalls, positions=range(len(models)),
                             showmeans=True, showmedians=True, showextrema=False)
        for i, pc in enumerate(parts['bodies']):
            pc.set_facecolor(c_model[models[i]]); pc.set_alpha(0.6)
        parts['cmeans'].set_color('#C62828'); parts['cmedians'].set_color('#1B5E20')
        for i, m in enumerate(models):
            s = np.maximum(-errors[mkt][m], 0)
            s_sorted = np.sort(s)[::-1]
            k = max(1, int(0.05 * len(s)))
            cvar95 = s_sorted[:k].mean()
            ax.annotate(f'CVaR$_{{95}}$={cvar95:.3f}', xy=(i, cvar95),
                       fontsize=8, ha='center', va='bottom', color='#C62828', fontweight='bold')
        ax.set_xticks(range(len(models))); ax.set_xticklabels(models, fontsize=11)
        ax.set_ylabel('Shortfall $s = \\max(\\tilde{H} - V_T, 0)$', fontsize=11)
        ax.set_title(f'{mkt.upper()}: Shortfall Distribution', fontsize=13, fontweight='bold')
        ax.grid(True, alpha=0.3, axis='y')
    fig.suptitle('(9e) Shortfall Distribution: Violin Plots with CVaR$_{95}$',
                 fontsize=15, fontweight='bold', y=1.02)
    fig.tight_layout()
    fig.savefig(os.path.join(save_dir, 'comparison_violin_shortfall.png'), dpi=200, bbox_inches='tight')
    plt.show()

    # ── FIGURE 9f: Grouped Metric Bar Chart ──
    fig, axes = plt.subplots(1, 3, figsize=(20, 6))
    x = np.arange(len(models)); width = 0.35
    ax = axes[0]
    for i, mkt in enumerate(markets):
        maes = [np.abs(errors[mkt][m]).mean() for m in models]
        ax.bar(x + i*width - width/2*(len(markets)-1), maes, width,
               label=mkt.upper(), color=c_market[mkt], alpha=0.85, edgecolor='black')
    ax.set_xticks(x); ax.set_xticklabels(models, fontsize=11)
    ax.set_ylabel('MAE', fontsize=11)
    ax.set_title('Mean Absolute Error', fontsize=13, fontweight='bold')
    ax.legend(fontsize=10); ax.grid(True, alpha=0.3, axis='y')
    ax = axes[1]
    for i, mkt in enumerate(markets):
        shortfalls = [np.maximum(-errors[mkt][m], 0).mean() for m in models]
        ax.bar(x + i*width - width/2*(len(markets)-1), shortfalls, width,
               label=mkt.upper(), color=c_market[mkt], alpha=0.85, edgecolor='black')
    ax.set_xticks(x); ax.set_xticklabels(models, fontsize=11)
    ax.set_ylabel('Mean Shortfall', fontsize=11)
    ax.set_title('Mean Shortfall $\\mathbb{E}[s]$', fontsize=13, fontweight='bold')
    ax.legend(fontsize=10); ax.grid(True, alpha=0.3, axis='y')
    ax = axes[2]
    for i, mkt in enumerate(markets):
        probs = [(errors[mkt][m] >= 0).mean() for m in models]
        ax.bar(x + i*width - width/2*(len(markets)-1), probs, width,
               label=mkt.upper(), color=c_market[mkt], alpha=0.85, edgecolor='black')
    ax.set_xticks(x); ax.set_xticklabels(models, fontsize=11)
    ax.set_ylabel('$P(V_T \\geq \\tilde{H})$', fontsize=11)
    ax.set_title('Super-Hedging Success Rate', fontsize=13, fontweight='bold')
    ax.axhline(1.0, color='gray', linestyle=':', alpha=0.5)
    ax.legend(fontsize=10); ax.grid(True, alpha=0.3, axis='y')
    fig.suptitle('(9f) Key Metrics: GBM vs Heston across Models',
                 fontsize=15, fontweight='bold', y=1.02)
    fig.tight_layout()
    fig.savefig(os.path.join(save_dir, 'comparison_metrics_bars.png'), dpi=200, bbox_inches='tight')
    plt.show()

    # ── FIGURE 9g: Combined 6-Model Violin ──
    fig, ax = plt.subplots(figsize=(14, 7))
    all_data, all_labels, all_colours = [], [], []
    for mkt in markets:
        for m in models:
            all_data.append(errors[mkt][m])
            all_labels.append(f'{m}\n({mkt.upper()})')
            all_colours.append(c_market[mkt])
    positions = range(len(all_data))
    parts = ax.violinplot(all_data, positions=positions, showmeans=True,
                         showmedians=True, showextrema=False)
    for i, pc in enumerate(parts['bodies']):
        pc.set_facecolor(all_colours[i]); pc.set_alpha(0.55)
    parts['cmeans'].set_color('#C62828'); parts['cmedians'].set_color('#1B5E20')
    bp = ax.boxplot(all_data, positions=positions, widths=0.12,
                   patch_artist=True, showfliers=False, zorder=5)
    for i, patch in enumerate(bp['boxes']):
        patch.set_facecolor(all_colours[i]); patch.set_alpha(0.8)
    ax.axhline(0, color='black', linewidth=1.5, linestyle='--', alpha=0.6)
    ax.set_xticks(positions); ax.set_xticklabels(all_labels, fontsize=10)
    ax.set_ylabel('Terminal Error $e_T$', fontsize=12)
    ax.set_title('All Models x Markets: Terminal Error Distribution', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='y')
    if len(markets) >= 2:
        sep_x = len(models) - 0.5
        ax.axvline(sep_x, color='gray', linewidth=2, linestyle='-', alpha=0.4)
        ax.text(sep_x/2, ax.get_ylim()[1]*0.9, 'GBM', ha='center',
               fontsize=12, fontweight='bold', color=c_market['gbm'])
        ax.text(sep_x + len(models)/2, ax.get_ylim()[1]*0.9, 'HESTON', ha='center',
               fontsize=12, fontweight='bold', color=c_market.get('heston', '#8E24AA'))
    fig.suptitle('(9g) Grand Comparison: All Models under Both Markets',
                 fontsize=15, fontweight='bold', y=1.01)
    fig.tight_layout()
    fig.savefig(os.path.join(save_dir, 'comparison_grand_violin.png'), dpi=200, bbox_inches='tight')
    plt.show()

plot_all_comparisons(errors)
print('\nAll 7 comparison figures saved to Figures/')

In [ ]:
# Cell 19: Download all outputs as zip (includes GBM, Heston, cross_market)
import shutil
from google.colab import files

shutil.make_archive('outputs', 'zip', '.', 'outputs')
files.download('outputs.zip')